## 全文检索

In [2]:
from pymilvus import MilvusClient, DataType


# Define server address
SERVER_ADDR = "http://localhost:19530"

client = MilvusClient(uri = SERVER_ADDR)

schema = client.create_schema()

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True, auto_id=True) 
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=256, enable_analyzer=True)     # VARCHAR 是 Milvus 用于文本存储的字符串数据类型。设置 enable_analyzer=True 以允许 Milvus 对文本进行分词。
schema.add_field(field_name="sparse", datatype=DataType.SPARSE_FLOAT_VECTOR) # 保留用于存储内部生成的稀疏嵌入向量的字段，用于全文搜索操作。数据类型必须为 SPARSE_FLOAT_VECTOR 。

{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 256, 'enable_analyzer': True}}, {'name': 'sparse', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>}], 'enable_dynamic_field': False}

In [ ]:
# 构建用于全文搜索的 bm25 Function
from pymilvus import FunctionType,Function

bm25_function = Function(
    name = "text_bm25_emb",
    input_field_names=["text"],
    output_field_names=["sparse"],
    function_type=FunctionType.BM25
)

schema.add_function(bm25_function)

# 配置索引
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="sparse",
    index_type="SPARSE_INVERTED_INDEX",  # 修改为 SPARSE_INVERTED_INDEX
    metric_type="BM25",
    params={
        "inverted_index_algo": "DAAT_MAXSCORE",  # 修正拼写错误
        "bm25_k1": 1.2,       # 数值越高，专业术语词频在文档排名中的重要性越大。取值范围：[1.2, 2.0]。
        "bm25_b": 0.75 
    }
)

# 检查集合是否已存在，如果存在则删除
if client.has_collection("my_collection"):
    client.drop_collection("my_collection")
    print("已删除现有集合 'my_collection'")

# 创建集合collections
client.create_collection(
    collection_name="my_collection",
    schema=schema,
    index_params=index_params,
)

print("成功创建集合 'my_collection'")

2025-08-25 21:37:05,197 [ERROR][handler]: RPC error: [create_index], <MilvusException: (code=65535, message=only metric type can be passed when use AutoIndex)>, <Time:{'RPC start': '2025-08-25 21:37:05.192486', 'RPC error': '2025-08-25 21:37:05.197632'}> (decorators.py:140)
2025-08-25 21:37:05,198 [ERROR][_create_index]: Failed to create an index on collection: my_collection (milvus_client.py:182)


已删除现有集合 'my_collection'


MilvusException: <MilvusException: (code=65535, message=only metric type can be passed when use AutoIndex)>